We will compare a simple exmple of a Sequential model (one input. one output) generated by keras and using the functional API

## Introduction to functional API

In [2]:
from keras import Input, layers

input_tensor=Input(shape=(32,))
dense=layers.Dense(32, activation='relu')

output_tensor=dense(input_tensor)

In [4]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)


Epoch 1/10
1000/1000 [==============================] - 0s 248us/step - loss: 11.5884
Epoch 2/10
1000/1000 [==============================] - 0s 8us/step - loss: 11.5337
Epoch 3/10
1000/1000 [==============================] - 0s 10us/step - loss: 11.5190
Epoch 4/10
1000/1000 [==============================] - 0s 7us/step - loss: 11.5105
Epoch 5/10
1000/1000 [==============================] - 0s 9us/step - loss: 11.5063
Epoch 6/10
1000/1000 [==============================] - 0s 7us/step - loss: 11.5013
Epoch 7/10
1000/1000 [==============================] - 0s 9us/step - loss: 11.4982
Epoch 8/10
1000/1000 [==============================] - 0s 8us/step - loss: 11.4956
Epoch 9/10
1000/1000 [==============================] - 0s 10us/step - loss: 11.4931
Epoch 10/10
1000/1000 [==============================] - 0s 37us/step


## Multi input models

In [33]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# The text input is a variable-length sequence of integers. Note that you can optionally 
# name the inputs
text_input = Input(shape=(None,), dtype='int32', name='text')

#Embeds the inputs into a sequence of vectors of size 64
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

# Encodes the vectors in a single vector via an LSTM
encoded_text = layers.LSTM(32)(embedded_text)

# Now, the same process (with different layers) for the question
question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question')

embedded_question = layers.Embedding( question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# Concatenates the encoded question and the encoded text
concatenated = layers.concatenate([encoded_text, encoded_question],
                                  axis=-1)

# adds a softmax classifier on top
answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated)

# At model instantiation, it's necessary to specify the two inputs and the output
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])




To feed the model we can go with one of two ways: feed a list of nmpy arrrays as inputs or feed it a dictionary that map input names to numpy arrays (need to give names to inputs).

In [34]:
import numpy as np

num_samples = 1000
max_length = 100

# generates dummy numpy data 
text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))
# answers are one-hot encoded, not integers
answers = np.random.randint(0, 1,
                            size=(num_samples, answer_vocabulary_size))
# Fitting using a list of inputs
model.fit([text, question], answers, epochs=10, batch_size=128)

# Fitting using a dictionary
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)




Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 1s 634us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 1s 624us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 1s 623us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
1000/1000 [==============================] - 1s 637us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 1s 630us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 1s 627us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
1000/1000 [==============================] - 1s 662us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
1000/1000 [==============================] - 1s 661us/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
1000/1000 [================

## Multi Output models 

In [35]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# we are giving name to the output ayers, so late we can specify which loss
# function belongs to which output layer
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])



In [36]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})


In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

# In this case we are specifying which loss functions should be applied to each output layers, 
# but it can happen that when trying to minimize one loss function we end up increasing another
# to prevet this we can assign weights to each loss function

model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})


## Diracted acyclic graphs of layers 